## Lambda Expressions

The following example illustrates the usage of lambda expressions and their corresponding closure objects within a __target__ region.

In CASE 1, a lambda expression is defined inside a __target__ construct that implicitly maps the structure 	extit{s}. Inside the construct, the lambda captures (by reference) the corresponding _s_, and the resulting closure object is assigned to _lambda1_.  When the call operator is invoked on _lambda1_, the captured reference to _s_ is used in the call.  The modified _s_ is then copied back to the host device on exit from the __target__ construct.

In CASE 2, a lambda expression is instead defined before the __target__ construct and captures (by copy) the pointer _sp_. A __target__ __data__ construct is used to first map the structure, and then the __target__ construct implicitly maps the closure object referenced by _lambda2_, a zero-length array section based on the structure pointer _sp_, and a zero-length array section based on the captured pointer in the closure object. The implicit maps result in attached pointers to the corresponding structure. The call for _lambda2_ inside the __target__ construct will access _sp->a_ and _sp->b_ from the corresponding structure.

CASE 3 is similar to CASE 2, except _s_ is instead captured by reference by the lambda expression. As for CASE 2, the structure is first mapped by an enclosing __target__ __data__ construct, and then the __target__ construct implicitly maps _s_ and the closure object referenced by _lambda3_. The effect of the map is to make the the call for _lambda3_ refer to the corresponding _s_ inside the __target__ construct rather than the original _s_.

In CASE 4, the program defines a static variable _ss_ of the same structure type as _s_. While the body of the lambda expression refers to _ss_, it is not captured. In order for _lambda4_ to be callable in the __target__ region, the reference to _ss_ should be to a device copy of _ss_ that also has static storage. This is achieved with the use of the __declare__ __target__ directive.  Inside the __target__ construct, all references to _ss_, including in the _lambda4()_ call, will refer to the corresponding _ss_ that results from the __declare__ __target__ directive. The __always__ modifier is used on the __map__ clause to transfer the updated values for the structure back to the host device.

In [ ]:
//%compiler: clang
//%cflags: -fopenmp

/*
* name:       lambda_expressions.1
* type:       C++
* version:   omp_5.0
*/
#include <iostream>
using namespace std;

struct S { int a; int b; };

int main()
{

// CASE 1 Lambda defined in target region

   S s = S {0,1};

   #pragma omp target
   {
      auto lambda1 = [&s]() { s.a = s.b * 2; };
      s.b += 2;
      lambda1(); // s.a = 3 * 2
   }
   cout << s.a << " " << s.b << endl; //OUT 6 3

// CASE 2 Host defined lambda, Capture pointer to s

   s = {0,1};
   S *sp = &s;
   auto lambda2 = [sp]() {sp->a = sp->b * 2; };

   // closure object's sp attaches to corresponding s on target
   // construct
   #pragma omp target data map(sp[0])
   #pragma omp target
   {
      sp->b += 2;
      lambda2();
   }
   cout << s.a << " " << s.b << endl; //OUT 6 3

// CASE 3 Host defined lambda, Capture s by reference

   s = {0,1};
   auto lambda3 = [&s]() {s.a = s.b * 2; };

   // closure object's s refers to corresponding s in target
   // construct
   #pragma omp target data map(s)
   #pragma omp target
   {
      s.b += 2;
      lambda3();
   }
   cout << s.a << " " << s.b << endl; //OUT 6 3

// CASE 4 Host defined lambda, references static variable

   static S ss = {0,1};
   #pragma omp declare target enter(ss)
   auto lambda4 = [&]() {ss.a = ss.b * 2; };

   #pragma omp target map(always,from:ss)
   {
      ss.b += 2;
      lambda4();
   }
   cout << ss.a << " " << ss.b << endl; //OUT 6 3

   return 0;
}